In [62]:
import gzip
import math
import random
from collections import defaultdict
import json
import ast
from collections import Counter
from operator import itemgetter
import numpy as np

import matplotlib.pyplot as plt
import random
from random import choices
random.seed(10)

In [35]:
user_list = []
f = open('australian_users_items_double_quotes.json', encoding="utf8")
for line in f:
       
    cur_json = json.loads(line)
    user_list.append(cur_json)

### Create train and test without considering playtime

In [36]:
user_item_list = []
for user in user_list:
    for item in user['items']:
        user_item_list.append((user['user_id'],item['item_id']))

In [38]:
random.shuffle(user_item_list)

In [39]:
train = user_item_list[:4000000]
test = user_item_list[4000000:][:500000]

In [40]:
print(f"Number of train is {len(train)}")
print(f"Number of test is {len(test)}")

Number of train is 4000000
Number of test is 500000


### Add false interaction to test data

In [41]:
#### Create dictionary
user_set = set([i[0] for i in user_item_list])
item_set = set([i[1] for i in user_item_list])
user_per_item = defaultdict(set)
item_per_user = defaultdict(set)


In [42]:
for user_item in user_item_list:
    user,item = user_item[0],user_item[1]
    user_per_item[item].add(user)
    item_per_user[user].add(item)
    

In [67]:
random.seed(10)
test_neg_user =  choices(list(user_set),k = 500000)

In [73]:
#### Maybe consider the distribution(popularity) of the game when sampling
random.sample(item_set,k = 1)

C:\Users\steve\AppData\Local\Temp\ipykernel_19752\2058216400.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(item_set,k = 1)


['359850']

In [83]:
item_per_user.keys()

dict_keys(['76561197977174002', 'Thisbananaistastyfam', '76561198085679048', 'ohhhhyeesss', '76561198011244140', 'Bowmore', '76561198049511816', '76561198144416667', '76561198073158808', '76561198080248619', '76561198004058300', 'MattyMarshun', '76561198080702354', '76561198067127934', 'astarlunne', '76561198034822832', 'inven', 'FallingPotato', 'merdehappens', '76561198087578204', '76561198063572252', 'innocentguy', '76561198083515632', 'Enderkills0909', 'thestreetsshallrunwhite', '76561198054591317', '9998886665555', '76561198043516885', '76561198048376894', 'Gotemlostinme', '76561198052993493', '76561198091153239', '76561198092086266', '76561198065080939', 'zeejackass', '76561198046042632', 'De-AK', 'probotsteve', 'kurbi-wan_kenobi', '76561198042330110', 'DeployableDuck', 'brendon1555', 'yamaaaaa', 'boxofcrits', '76561198067777635', 'hoarfrostkid', '3021Daniel', 'SirTrollbuscus', '76561198102667143', '76561198085895776', 'purpledragonfun', '76561198045942232', 'balbahutog27', 'IceSu

In [88]:
random.seed(10)
test_neg_entries = []
for user in test_neg_user:
    item_candidate = random.sample(item_set,k = 1)[0]
    temp = item_per_user[user]
    while item_candidate in temp:
        item_candidate = random.sample(item_set,k = 1)[0]
    test_neg_entries.append((user,item_candidate))


C:\Users\steve\AppData\Local\Temp\ipykernel_19752\1083373277.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  item_candidate = random.sample(item_set,k = 1)[0]
C:\Users\steve\AppData\Local\Temp\ipykernel_19752\1083373277.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  item_candidate = random.sample(item_set,k = 1)[0]


In [90]:
len(test_neg_entries)

500000

In [91]:
test_total = test+test_neg_entries

In [127]:
len(test_total)

1000000

In [128]:
test_total_y = [1]*500000+[0]*500000

### Baseline Model (Popularity Based)

In [94]:
game_list = []
for i in train:
    user,item = i[0],i[1]
    game_list.append(item)

In [97]:
game_count = Counter(game_list)
sorted_game = sorted(game_count.items(),key=itemgetter(1),reverse = True)

In [109]:
game_list_sorted = [i[0] for i in sorted_game]
############## DEFINE POPULAR GAME ################
popular_games = game_list_sorted[:5000]

In [113]:
baseline_pred = []
for i in test_total:
    user,item = i[0],i[1]
    if item in popular_games:
        baseline_pred.append(1)
    else:
        baseline_pred.append(0)

In [131]:
baseline_acc = sum(1 for x,y in zip(test_total_y,baseline_pred) if x == y) / len(baseline_pred)
baseline_acc

0.764519

### Similarity Based Model